In [2]:
from my_libs_py3 import *
pd.set_option("display.max_rows",100)
import bson
import cudf


Bad key "text.kerning_factor" on line 4 in
/home/ken/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/home/ken/miniconda3/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [ ]:
df = cudf.DataFrame({"a":{1, 2, 3, 4, 5}})

data = data.drop("_id",axis=1)

data["Refresh_Date"] = data["Refresh_Date"].apply(lambda x: x.strftime("%Y-%m-%d"))


In [40]:
mongod = mongo("all_symbol","screener")
## This line gets the max refresh_date
tos = pd.DataFrame(mongod.conn.table.find({},{"Refresh_Date":1}).sort("Refresh_Date",-1).limit(1))["Refresh_Date"].iloc[0]
all_ticker = pd.DataFrame(mongod.conn.table.find({"Refresh_Date":tos}))
all_ticker = all_ticker[all_ticker.Volume > all_ticker.Volume.quantile(0.5)]
all_ticker = all_ticker[all_ticker["Institutional Ownership"] > all_ticker["Institutional Ownership"].quantile(0.5)]

all_ticker = all_ticker.Ticker.to_list()


Connection Successful


In [62]:
def pair_trade_smaple():
    mongod = mongo("all_symbol","pair_trade_sharp_2021")
    candid = pd.DataFrame(mongod.conn.table.find({"Sharp_Ratio":{"$exists":True}}))
    top_return = candid.describe().loc["75%","Avg_Return"]
    top_sharp = candid.describe().loc["75%","Sharp_Ratio"]

    pipeline = [
        {"$match":{"End_Value":{"$gt":2000},"Sharp_Ratio":{"$gt":top_sharp},"Avg_Return":{"$gt":top_return}}},
        {"$sample":{"size":10}}
    ]

    candid = pd.DataFrame(mongod.conn.table.aggregate(pipeline))

    return candid

In [64]:
pair_trade_smaple()

Connection Successful


,_id,Ticker_1,Ticker_2,End_Value,Avg_Return,Sharp_Ratio,Min_Return,Max_Return,Refresh_Date
0,612c5e83ffb50dc4b25489be,ALRM,B,"2,240.4159",0.0005,0.0649,-0.0389,0.0413,2021-08-22 00:00:00+00:00
1,6122eed5ffb50dc4b24dd0c6,AAT,GNW,"2,544.0665",0.0010,0.0680,-0.0586,0.1562,2021-08-22 00:00:00+00:00
2,6126496dffb50dc4b2502fe2,ADSK,TIG,"2,577.8298",0.0010,0.1031,-0.0495,0.0379,2021-08-22 00:00:00+00:00
3,612d09b4ffb50dc4b255023a,AMBC,RTPYU,"2,167.0800",0.0007,0.1373,-0.0204,0.0282,2021-08-22 00:00:00+00:00
4,61268243ffb50dc4b25057bc,ADTN,HMHC,"2,442.3220",0.0008,0.0748,-0.0694,0.0511,2021-08-22 00:00:00+00:00
5,61244c3effb50dc4b24ec414,ACIW,SI,"2,755.4998",0.0013,0.1275,-0.0140,0.1151,2021-08-22 00:00:00+00:00
6,612406f6ffb50dc4b24e9344,ACC,UNIT,"2,337.6120",0.0006,0.0857,-0.0287,0.0327,2021-08-22 00:00:00+00:00
7,612b22c9ffb50dc4b253a596,ALGN,CORT,"2,448.4300",0.0008,0.0708,-0.0634,0.0589,2021-08-22 00:00:00+00:00
8,612b2521ffb50dc4b253a738,ALGN,DOC,"2,431.3901",0.0008,0.0777,-0.0634,0.0589,2021-08-22 00:00:00+00:00
9,6123215bffb50dc4b24df3ca,ABCB,RDWR,"2,133.5067",0.0003,0.1111,-0.0129,0.0301,2021-08-22 00:00:00+00:00


In [42]:
trade = self_pair_trade("ADP","APR").iloc[-1]

All price data of Close is actually Adj Close
Connection Successful
ADP
('Finished', 'ADP')
All price data of Close is actually Adj Close
Connection Successful
APR
('Finished', 'APR')
All price data of Close is actually Adj Close
('Finished', 'ADP')
All price data of Close is actually Adj Close
('Finished', 'APR')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [8]:
trade.iloc[-1]["total_value"]

2571.852227

In [24]:
np.log(trade["total_value"].shift(-1)/trade["total_value"]).mean()

0.0010140289098981633

In [23]:
trade["total_value"].head(7)

TimeStamp
2020-08-26   2,000.0000
2020-08-27   2,000.0000
2020-08-28   2,000.0000
2020-08-31   2,000.0000
2020-09-01   2,000.0000
2020-09-02   2,014.9500
2020-09-03   1,965.7615
Name: total_value, dtype: float64

-----

### Trade Action

In [44]:
ticker1="ADP"
ticker2="APR"

today_trade = self_pair_trade(ticker1,ticker2,method = "realtimeday").iloc[-1]

log = get_pair_trade_log(ticker1,ticker2)

# day_diff = log["TimeStamp"].astimezone(mytz) - datetime.now(mytz)


if len(log) == 0:
    current_size1 = 0
    current_size2 = 0
else:
    log = log.iloc[-1]
    current_size1 = log["size1"]
    current_size2 = log["size2"]


new_size1 = today_trade["size1"]
new_size2 = today_trade["size2"]


trade_size1 = new_size1 - current_size1
trade_size2 = new_size2 - current_size2


try:
    td_trade = order_equity(order_type.LIMIT)

    ## need to trade the short sell first to test whether it's short
    if trade_size1 != 0:
        send_email(title="!Important! Pair Trade Order Placing for %s and %s"%(ticker1,ticker2))
        if trade_size1 < 0:
            td_trade.place(ticker1,trade_size1)
            
            td_trade.place(ticker2,trade_size2)
        if trade_size1 > 0:
            td_trade.place(ticker2,trade_size2)
            td_trade.place(ticker1,trade_size1)
        send_email("Trade process done.",title="!Important! Pair Trade Order Placed for %s and %s"%(ticker1,ticker2))
    else:
        print("No trade, skiped for %s and %s"%(ticker1,ticker2))
except Exception as e:
    send_email(str(e),title="!Important! Pair Trade Place Order Error")

All price data of Close is actually Adj Close
('Finished', 'ADP')
All price data of Close is actually Adj Close
('Finished', 'APR')
All price data of Close is actually Adj Close
('Finished', 'ADP')
All price data of Close is actually Adj Close
('Finished', 'APR')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Connection Successful


In [3]:
mongod = mongo("all_symbol","pair_trade_sharp_2021_500")
candid = pd.DataFrame(mongod.conn.table.find({"Sharp_Ratio":{"$exists":True}}))

Connection Successful


In [7]:
candid[(candid.Ticker_1 == "ACA")&(candid.Ticker_2 == "DK")]

,_id,Ticker_1,Ticker_2,End_Value,Avg_Return,Sharp_Ratio,Min_Return,Max_Return,Refresh_Date
13355,612facb382f24db0e85f9777,ACA,DK,587.9126,0.0007,0.0707,-0.0254,0.0955,2021-08-31 00:00:00+00:00


In [50]:
ticker1="ACA"
ticker2="DK"
self_pair_trade(ticker1,ticker2,method = "realtimeday",cash = 500)

All price data of Close is actually Adj Close
('Finished', 'ACA')
All price data of Close is actually Adj Close
('Finished', 'DK')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


,relative,relative_mv,stock1,stock2,slope,hedge_ratio,z_score,up,mid,low,buy_line,sell_line,cash,buying_power,margin1,margin2,size1,size2,holddays,total_value
TimeStamp,,,,,,,,,,,,,,,,,,,,
2020-09-08,-0.0008,0.0008,46.4246,13.4300,-0.0088,0.2893,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-09,-0.0008,0.0008,46.4445,12.8400,-0.0088,0.2893,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-10,-0.0008,0.0008,44.4217,12.1500,-0.0088,0.2829,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-11,-0.0008,0.0008,44.3022,12.0600,-0.0088,0.2799,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-14,-0.0008,0.0008,43.4950,12.2000,0.3151,0.2781,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-27,-0.0021,0.0027,52.2300,16.9900,2.1737,0.3158,-1.5822,-1.2590,-1.6421,-2.0251,-1.2500,1.2500,215.5726,111.1126,0.0000,104.4600,8,-2,0,599.4326
2021-08-30,-0.0018,0.0026,51.4900,16.9200,2.0676,0.3162,-1.4615,-1.2767,-1.6478,-2.0189,-1.2500,1.2500,215.5726,111.1126,0.0000,104.4600,8,-2,1,593.6526
2021-08-31,-0.0025,0.0027,50.8200,17.1100,2.1593,0.3167,-1.7166,-1.3257,-1.6828,-2.0400,-1.2500,1.2500,215.5726,111.1126,0.0000,104.4600,8,-2,2,587.9126


In [23]:
def cancel_order(orderid):
    return client.cancelOrder(277216702,orderid)

In [66]:
mongod = mongo("all_symbol","pair_trade_sharp_2021_500")
candid = pd.DataFrame(mongod.conn.table.find({"Sharp_Ratio":{"$exists":True}}).sort("Sharp_Ratio",-1).limit(10))

Connection Successful


In [67]:
candid

,_id,Ticker_1,Ticker_2,End_Value,Avg_Return,Sharp_Ratio,Min_Return,Max_Return,Refresh_Date
0,612e8add82f24db0e85ec8d1,A,BWXT,606.2036,0.0008,0.1951,-0.0172,0.0214,2021-08-31 00:00:00+00:00
1,612e8e3082f24db0e85ecb21,A,CTSH,559.9013,0.0005,0.1852,-0.0133,0.0137,2021-08-31 00:00:00+00:00
2,612e937d82f24db0e85eceeb,A,GILD,591.5851,0.0007,0.1842,-0.0147,0.0210,2021-08-31 00:00:00+00:00
3,612e9a2c82f24db0e85ed39d,A,MDLA,623.0133,0.0009,0.1698,-0.0172,0.0228,2021-08-31 00:00:00+00:00
4,612e8c1e82f24db0e85ec9ab,A,CHD,519.4292,0.0002,0.1667,-0.0023,0.0134,2021-08-31 00:00:00+00:00
5,612e9ab182f24db0e85ed3f9,A,MKC,521.2508,0.0002,0.1667,-0.0060,0.0083,2021-08-31 00:00:00+00:00
6,612e925c82f24db0e85ece1b,A,FIXX,636.0371,0.0010,0.1667,-0.0208,0.0247,2021-08-31 00:00:00+00:00
7,612e861b82f24db0e85ec587,A,ACAD,607.3677,0.0008,0.1633,-0.0146,0.0531,2021-08-31 00:00:00+00:00
8,612e97f682f24db0e85ed20d,A,KPTI,613.8159,0.0008,0.1633,-0.0208,0.0257,2021-08-31 00:00:00+00:00
9,612e9c4d82f24db0e85ed519,A,NLS,605.3867,0.0008,0.1633,-0.0184,0.0254,2021-08-31 00:00:00+00:00


In [30]:
log = get_pair_trade_log("AA_RUN")

Connection Successful


In [46]:
log

,_id,TimeStamp,Ticker1,Ticker2,size1,size2,Price1,Price2
0,61322b143d0072716eb06a2e,2021-09-03 07:03:00.887000+00:00,AA,RUN,5.0000,-5.0000,None,None
1,612f95a73cf3b515f48eecf0,2021-09-01 08:00:55.168000+00:00,AA,RUN,19.0000,-22.0000,None,None


In [32]:
log["size1"].sum()

24.0

In [33]:
log["size2"].sum()

-27.0

In [29]:
current_size1

19.0

In [52]:
get_pair_trade_log("ACC_AMH")

Connection Successful


,_id,TimeStamp,Ticker1,Ticker2,size1,size2,Price1,Price2
0,61322b403d0072716eb06a32,2021-09-03 07:03:44.540000+00:00,ACC,AMH,-5.0000,4.0000,None,None
1,6130f572e4f81f0169fc362c,2021-09-02 09:01:54.048000+00:00,ACC,AMH,-5.0000,4.0000,None,None


In [47]:
get_pair_open_opsition()

Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful


['ACA_GTHX', 'ACA_DK', 'AA_RUN', 'ACC_AMH']

In [43]:
today_trade = self_pair_trade(ticker1,ticker2,method = "realtimeday",cash = 500).iloc[-1]

ticker_combo = ticker1+"_"+ticker2
log = get_pair_trade_log(ticker_combo)

# day_diff = log["TimeStamp"].astimezone(mytz) - datetime.now(mytz)


if len(log) == 0:
    current_size1 = 0
    current_size2 = 0
else:
    
    current_size1 = log["size1"].sum()
    current_size2 = log["size2"].sum()


new_size1 = today_trade["size1"]
new_size2 = today_trade["size2"]


trade_size1 = new_size1 - current_size1
trade_size2 = new_size2 - current_size2

All price data of Close is actually Adj Close
('Finished', 'AA')
All price data of Close is actually Adj Close
('Finished', 'RUN')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Connection Successful


In [44]:
trade_size1

-19.0

In [45]:
trade_size2

22.0

### DB manipulation

In [23]:
ticker = ticker1+"_"+ticker2
mongod = mongo("pair_trade_log",ticker)
mongod.conn.table.drop()

Connection Successful


In [4]:
mongod = mongo("pair_trade_log","pair_trade_log")

mongod.conn.db.list_collection_names()

Connection Successful


['AA_RUN']

In [24]:
mongod = mongo("all_symbol","td_not_shortable")
mongod.conn.table.insert_one({"TimeStamp":datetime.now(),"symbol":"APR"})

Connection Successful


In [25]:
pd.DataFrame(mongod.conn.table.find())

,_id,TimeStamp,symbol
0,612d38fc849e5662f18dc997,2021-08-30 13:01:00.435000+00:00,APR


In [19]:
client.accountsDF().loc[0,"currentBalances.availableFunds"]

441.99